# Get the data

In [ ]:
import pandas
import numpy as np

In [ ]:
dataframe = pandas.read_csv('data.csv', encoding='ISO-8859-1')

dataframe_counts = np.copy(dataframe.values[:,1:])
dataframe_counts[dataframe_counts == '..'] = 0 # Use zeros to indicate missing data
dataframe_counts = dataframe_counts.astype(np.int)

dataframe.values[:,1:] = np.copy(dataframe_counts) # Replace pesky string integers

data_point_count = dataframe_counts.shape[0]
median_count = np.median(dataframe_counts[dataframe_counts > 0])

years = dataframe.columns[1:]
years = [int(s) for s in years]

names = dataframe.values[:,0]

# Preprocess the data

In [ ]:
dataframe_counts_pp = dataframe_counts / median_count # Scaled version of the data

train_data_point_count = int(0.8 * data_point_count)
val_data_point_count = data_point_count - train_data_point_count

train_dataframe = dataframe_counts_pp[:train_data_point_count,:]
val_dataframe = dataframe_counts_pp[train_data_point_count:,:]

train_names = names[:train_data_point_count]
val_names = names[train_data_point_count:]

# Define the model

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D

In [ ]:
keras.backend.clear_session() # In case you want to get rid of models in RAM

In [ ]:
BATCH_SIZE = 32
DATA_DIMENSIONALITY = 1 # Per (lastname, year) there is one piece of data: the count
input_shape = (None, DATA_DIMENSIONALITY)

## Mean Absolute Error (MAE)

In [ ]:
model = Sequential([LSTM(64, return_sequences=True, input_shape=input_shape),
                    LSTM(64, return_sequences=True),
                    Dense(1)])

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=['mae'])

model.summary()

# Creating a custom data generator
Long story short, we want to be able to split our data up into random windows, but with respect to the window's size and locations. This is not feasible to create as a dataset (keeping all possible subsets in RAM is not a good idea). We therefore need a custom data generator.

In [ ]:
# Reference:
# https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, min_seq_length=3, max_seq_length = None):
        self.data = data
        self.indices = list(range(data.shape[0]))
        self.random_indices = None
        
        self.seq_lengths = data.shape[1]
        self.min_seq_length = min_seq_length
        self.max_seq_length = max_seq_length
        
        self.on_epoch_end()

        if max_seq_length is None:
            self.max_seq_length = self.seq_lengths - 1

    def __len__(self):
        res = len(self.indices) // BATCH_SIZE
        if res == 0 and len(self.indices) > 0:
            res = 1
        return res

    def __getitem__(self, index):
        random_indices_batch = self.random_indices[index * BATCH_SIZE : (index + 1) * BATCH_SIZE]
        window_size_batch = np.random.randint(self.min_seq_length, high=self.max_seq_length)
        window_offsets_batch = np.random.randint(0, high=(self.seq_lengths - window_size_batch), size=BATCH_SIZE)
        
        X = np.zeros((BATCH_SIZE, window_size_batch, DATA_DIMENSIONALITY))
        Y = np.zeros((BATCH_SIZE, window_size_batch, DATA_DIMENSIONALITY))

        for i in range(BATCH_SIZE):
            i_r = random_indices_batch[i]
            w_o = window_offsets_batch[i]
            
            start_index = w_o
            stop_index = w_o + window_size_batch
            
            X[i, :, :] = np.expand_dims(self.data[i_r, start_index:stop_index], axis=-1)
            Y[i, :, :] = np.expand_dims(self.data[i_r, start_index+1:stop_index+1], axis=-1)
        
        return X, Y

    def on_epoch_end(self):
        random_indices_length = np.maximum(BATCH_SIZE, len(self.indices))
        self.random_indices = np.random.randint(0, high=len(self.indices), size=random_indices_length)

# Fitting the model

Possible improvements:
* Implement checkpoints
* Implement early stopping

In [ ]:
dg_train = DataGenerator(train_dataframe, min_seq_length=20)
dg_val = DataGenerator(val_dataframe, min_seq_length=20)

history = model.fit(dg_train, validation_data=dg_val, epochs=200)

# Plot the results

In [ ]:
import matplotlib.pyplot as plt

plt.title("Mean Absolute Error (MAE)")
plt.plot(history.history['mae'], label='MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.legend(loc='upper right')

plt.show()

# Prediction vs reality

In [ ]:
print("Available names:")
print(", ".join(list(val_names)))

### Note
The model returns a sequence of values. Sometimes the magnitude of this sequence does not match the magnitude of the actual data. This can cause it to spuriously seem like the model is predicting an increase/decrease. We remedy this by looking at the difference between the last and the second last value in the output of the data, rather than just the last value.

In [ ]:
def predict_n_steps(input_data, n):
    predictions = np.zeros(n)
    current_window = input_data

    # NOTE: This portion predicts based on the last value of the network.
    # for i in range(prediction_count):
    #     current_window = model.predict(current_window)
    #     predictions[i] = current_window[0, -1, 0]

    # NOTE: This portion predicts based on the difference between
    # the last and second last values of the network.
    last_value = input_data[0, -1, 0]
    for i in range(n):
        current_window = model.predict(current_window)
        predicted_diff = current_window[0, -1, 0] - current_window[0, -2, 0]
        predictions[i] = last_value + predicted_diff
        last_value = predictions[i]
        
    return predictions

In [ ]:
name = "Sjöberg"
svensson_index = np.argwhere(val_names == name)[0][0]

svensson_data = np.copy(val_dataframe[svensson_index,0:])
svensson_data = np.expand_dims(svensson_data, axis=0)
svensson_data = np.expand_dims(svensson_data, axis=-1)

prediction_count = 11

predictions = predict_n_steps(svensson_data[:,:-prediction_count,:], prediction_count)

svensson_data *= median_count
predictions *= median_count

svensson_data = np.squeeze(svensson_data)

## Plot predictions

In [ ]:
prediction_years = list(range(years[-1] + 1 - prediction_count, years[-1] + 1))

plt.title("Number of people with the last name {}".format(name))
plt.plot(years, svensson_data, label='Actual')
plt.plot(prediction_years, predictions, label='Predicted')
plt.legend(loc='lower left')

plt.show()

# Make predictions for the future

In [ ]:
name = "Sjöberg"
svensson_index = np.argwhere(val_names == name)[0][0]

svensson_data = np.copy(val_dataframe[svensson_index,:])
svensson_data = np.expand_dims(svensson_data, axis=0)
svensson_data = np.expand_dims(svensson_data, axis=-1)

prediction_count = 11

predictions = predict_n_steps(svensson_data, prediction_count)

svensson_data *= median_count
predictions *= median_count

svensson_data = np.squeeze(svensson_data)

## Plot predictions for the future

In [ ]:
prediction_years = list(range(years[-1] + 1, years[-1] + 1 + prediction_count))

plt.title("Number of people with the last name {}".format(name))
plt.plot(years, svensson_data, label='Actual')
plt.plot(prediction_years, predictions, label='Predicted')
plt.legend(loc='lower left')

plt.show()

# Sidenote: finding a nice learning rate
In short: results do not seem to be very dependent on learning rate in this case

In [ ]:
model = Sequential([LSTM(64, return_sequences=True, input_shape=input_shape),
                    LSTM(64, return_sequences=True),
                    Dense(1)])

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=['mae'])

lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))

dg_train = DataGenerator(train_dataframe, min_seq_length=20)

history = model.fit(dg_train, epochs=100, callbacks=[lr_schedule])

In [ ]:
import matplotlib.pyplot as plt

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])
plt.xlabel("lr")
plt.ylabel("loss")
plt.show()